In [10]:
# Created by Balakrishna Vagvala and Sherrill Kirk R
# This code depends upon the packages and libraries listed in the next cell
# Date of creation: 12/1/2020
# Script purpose: this script processes the OSU CCAL Lab delivered data, compares with expected fields file and make ready to be uploaded to database.

In [11]:
# We use openpyxl to load excel work book, this will help in having our current data as new sheet in the workbook
# we use shutil to copy a file from it's location to a desired location,
# this will help in copy pasting our original file to make changes on it
# We use difflib library and import sequencematcher function, this will help us in comparing strings and give us a match ratio
# We use numpy library and import it as np to make numerical operations for our program
# we use pandas library and immport it as np to make data operations for our lab data
# we use time library to import curent time of our function
# we use OS library for different functions such as writing data to file and saving files
# We use glob library and its glob function finds all the pathnames matching a specified pattern

In [12]:
#importing pandas for data operations, numpy for numerical operations. time is used to have present time.
import pandas as pd
import numpy as np
import time
#sequence matcher is used to compare strings and openpyxl is used to write workbooks.
from difflib import SequenceMatcher
from openpyxl import load_workbook
from re import search
from shutil import copyfile
#importing reading functions and declaring the location of our file.
import os, glob

In [13]:
# mounting google drive for google collab, not needed for native IDE's
# from google.colab import drive
# drive.mount('/content/drive')

In [14]:
########################
#Variables to be defined
########################

#file to check the field names and change if needed
crosswalkpath = r"D:\Bala\OSU CCAL\osuccal_expectedfields.xlsx"

# enter the output path name
currentoutputpath= r"D:\Bala\OSU CCAL\combined_output\\"

#path of the folder which contains all the test america lab files
inputpath =r"D:\Bala\OSU CCAL\input"

#Wild Card Syntax used to define the files to be processed 
wildCardSyntax = "**.xlsx"



#Output file name suffix for the Logfile and Subset/Appended file post processing
outPutFileSuffix = "OSUCCAL"


In [15]:
# runtime for file

timestr = time.strftime("_%Y%m%d_%H%M")
runtimenow = timestr

# the opening the log file
logfile= open(currentoutputpath+outPutFileSuffix+runtimenow+".txt","w+")


listofdataframes = []

#Path where all the excel files are residing to be processed
all_files = glob.glob(os.path.join(inputpath, wildCardSyntax))


for  no,f in enumerate(all_files):
    df = pd.read_excel(f, sheet_name = 1)
    #file to check crosswalks, i.e field names, this is an excel file's path
    
    #toread crosswalk file
    crosswalk = pd.read_excel(crosswalkpath)
    #file to process , this is an excel file's path
#     data = "D:\Bala\OSU CCAL\input\GRKO_032620_changed_names.xlsx"

    
    
    filename= os.path.basename(f)
    print(filename)
    sep = '.'
    filename = filename.split(sep, 1)[0]

    

    firstline=[]
    firstline.append("File Being Processed is: " + filename+ "\n \n \n#########\n\n" )
    #toread the lab file, we mention sheet name of excel file in the second argument, here GRKO Data is our sheet name in excel file by lab
    # osudata = pd.read_excel(data, 'GRKO Data')
    #to specify sheet number instead of sheet name
#     osudata = pd.read_excel(data, sheet_name = 1)

    #here we process our data so that we have just the field names and field values in our dataframe
    osudata = df
    df = df.iloc[2:]
    df = df.rename(columns=df.iloc[0])
    df.reset_index(inplace=True)
    df = df.iloc[1:,1:]
    

    # to remove \n and \r from lab delivareables, they are influencing our data and output, so we are removing them

    dfStrip = osudata.replace('\n',' ', regex=True)

    dfStrip2 = dfStrip.replace('\r','', regex=True)

    df = dfStrip2

    df = df.iloc[2:]

    df = df.rename(columns=df.iloc[0])

    df.reset_index(inplace=True)

    df = df.iloc[1:,1:]

    osufields = list(df.columns)
    labfields = crosswalk['NativeDeliveryFIelds'].tolist()
    expectedfields = crosswalk['CrossWalkedFields'].tolist()
    # this cell is to check for coloumns and export irregularities into warning list
    
    matchingfields =[]
    lenghtoffields = []
    morethanninety = []
    lessthanninety = []
    newfields=[]
    duplicatefields=[]
    missedfields = []
    p=''
    q=''
    x=''
    if len(osufields)>len(labfields):
        lenghtoffields.append('\n\nThis '+filename+' file has new columns.\n')

    if len(osufields)<len(labfields):
        lenghtoffields.append('This '+filename+' file has lesser columns than expected.\n')

    duplicatefields.append ('\n\n#####\n\nFields with duplicate field names \n') 
    missedfields.append ('\n\n######\n\nFields which are expected, but not in our data\n')
    matchingfields.append ('\n\n#####\n\nFields that matched as expected') 
    morethanninety.append ('\n\n#####\n\nFields without 100% match but had more than 90% Match\n') 
    lessthanninety.append ('\n\n#####\n\nFields with less than 90% match but had more than 80% Match\n') 
    newfields.append ('\n\n#####\n\nNew Fields , these are not found in crosswalk and have been removed from data \n') 

    for i in osufields:
        if i == 'Date': 
            x = osufields.index(i)
            osufields[x] = osufields[x-1] +(" ") +osufields[x]
            continue 
    for j in labfields:
        if j == 'Date':
            x = labfields.index(j)
            labfields[x] = labfields[x-1] +(" ") + labfields[x]
    df.columns = osufields

    for i in expectedfields: 
        for j in osufields:
            k = i+"."+"1"
            if search(k, j):
                print(k)
                duplicatefields.append("Warning: "+i+ " has a duplicate field \n")
                break

    for i in osufields:     
        if i in labfields:
            matchingfields.append("column: "+ i +"  has sucessfully matched \n")
            x = osufields.index(i)
            y = labfields.index(i)        
            osufields[x] = expectedfields[y]
            continue      

        else:
            result = ""
            count = 0
            ratio = 0  

            for words in labfields:
                ratio = SequenceMatcher(None,i,words).ratio()
                if ratio > count:
                    count = ratio
                    result =words         

            if count > 0.9:
                x = osufields.index(i)
                y = labfields.index(result)        
                morethanninety.append("Warning: "+osufields[x]+" was crosswalk to "+ result + " has matched with more than 90% accuracy \n")
                osufields[x] = expectedfields[y]



            elif count> 0.8:
                x = osufields.index(i)
                y = labfields.index(result)
                lessthanninety.append("Warning: "+osufields[x]+" was crosswalk to "+ result + " has matched with more than 80% accuracy  \n")

                osufields[x] = expectedfields[y]



    # temp to check the missing columns in our processed lab deliverable 
    missedfields = []
    missedfields.append ('\n\n######\n\nFields which are expected, but not in our data\n')
    temp =  expectedfields.copy()
    for i in osufields:
        if i in temp:
            temp.remove(i) 
    for i in temp:
        missedfields.append("column: "+ i + " is expected but is not in our current data \n ")        

    df.columns = osufields

    for i in osufields:
        if i not in expectedfields:
            newfields.append("Warning: "+i+" is a new field, look into it\n")
            x = list(df.columns).index(i)
            del osufields[x]
            cols = [i for i in range(df.shape[1])]
            cols.remove(x)
            df = df.iloc[:,cols]


    df.columns = osufields
    # to reindex columns based on expected fields
    
    df = df.loc[:,~df.columns.duplicated()]
    
    df = df.reindex(expectedfields, axis=1)
    # this will drop columns with no values which are picked up from re-indexing
    df = df.dropna(axis=1)

    worked_files = firstline  + lenghtoffields+ morethanninety +lessthanninety + duplicatefields + missedfields+newfields + matchingfields
    for i in range(len(worked_files)):
        logfile.write(worked_files[i])
        
    count_row = df.shape[0]  # Gives number of rows
    count_col = df.shape[1]  # Gives number of columns

    logfile.write('\n\n######\nNumber of Rows = '+ str(count_row)+'\nNumber of Columns = '+ str(count_col)+'\nthis is end of the Log File for '+filename+'\n\n###########\n\n')
          

#     pd.set_option("display.max_rows", None, "display.max_columns", None)
#     df

    #saving our new csv file into a downloadable format. and it can be saved
    # df.to_csv( "data_fieldsprocessed"+timestr+".csv", index=False, encoding='utf-8-sig')
    #if you want an excel output
    # f = open
    # df.to_excel("pre-processed"+timestr+".xls", index=False, encoding='utf-8-sig')

    # to make a duplicate of our file and export our processed 
    # copyfile ('D:\Bala\OSU CCAL\input\GRKO_032620.xlsx' , './processed'+filename+'.xlsx')
    processed_path = (currentoutputpath+"\\"+filename+'_processed.xlsx')
    copyfile (f , processed_path)
#     data

    # rename our copied file and place it at the location we want

    # os.rename(r'D:\Bala\OSU CCAL\output\processed.xlsx', r'D:\Bala\OSU CCAL\output\OSUCCAL_processed_' + filename + '.xlsx')

    # enter the path of the processed named copy of the original file
    # processed_path = 'D:\Bala\OSU CCAL\output\OSUCCAL_processed__filename.xlsx'
    # to load our data frame into excel sheet
    book = load_workbook(processed_path)
    writer = pd.ExcelWriter(processed_path, engine = 'openpyxl')
    writer.book = book
    # enter the desired sheet name below
    sheetname = 'preprocessed' #enter your sheet name here in the space, we can have it as a prefix as you want
    df.to_excel(writer, sheet_name = sheetname)
    writer.save()
    writer.close()
    listofdataframes.append(df)

logfile.close() 

GRKO_032620.xlsx
GRKO_042820.xlsx
GRKO_071620.xlsx
ROMN_112219_121719.xlsx


In [16]:
combined_file = pd.concat( listofdataframes) 

combined_file.to_csv( currentoutputpath+"combined_csv of "+outPutFileSuffix+timestr+".csv", index=False, encoding='utf-8-sig')

In [17]:
#to read Excel file of the crosswalk
# data_xls = pd.read_excel(crosswalk,dtype=str, index_col=None)
# checkfile = data_xls
#sheet to speicy sheet number and index_col to have a default index or not.
#data_xls.to_csv('csvfile.csv', encoding='utf-8', index=False)

#     uncomment below line if its csv file, remove the '#' symbol
# all_files = glob.glob(os.path.join(path, "*TalStandard.csv"))
